In [0]:
df = spark.table("classes.gold.fato_consumo_agua")
display(df)

from pyspark.sql import functions as F

df_top10_imposto = (
    df.groupBy("regiao")
      .agg(F.max("valor_imposto_decimal").alias("maior_imposto"))
      .orderBy(F.desc("maior_imposto"))
      .limit(10)
      .withColumn(
          "maior_imposto",
          F.concat(
              F.lit("R$ "),
              F.regexp_replace(
                  F.regexp_replace(F.format_number("maior_imposto", 2), ",", "X"),
                  "\\.", ","
              )
          )
      )
      .withColumn(
          "maior_imposto",
          F.regexp_replace("maior_imposto", "X", ".")
      )
)

display(df_top10_imposto)

df_top10_imposto.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("classes.gold.top10_regioes_imposto_FORMATADO")